# 🧮 Building Analysis: Computational Tools for Compliance

This notebook demonstrates **practical computational tools** that our compliance agent uses to analyze buildings.

**Goal**: Show you the tools the agent has available for sophisticated building analysis.

**Building**: Real Vilamalla Industrial Complex (9 levels, 23 doors, 102 walls)


## 🚀 Step 1: Load Real Building Data

Let's load the Vilamalla building data that our agent works with:

In [ ]:
# Load building data and computational tools
import sys
sys.path.append('..')

from scripts.load_building_data import load_vilamalla_building
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

# Load the enhanced building data
print("🏗️ Loading Vilamalla Industrial Complex...")
loader = load_vilamalla_building()

print(f"✅ Building Data Loaded:")
print(f"   Project: {loader.metadata.get('project_name')}")
print(f"   Levels: {len(loader.levels)}")
print(f"   Rooms: {len(loader.all_rooms)}")
print(f"   Doors: {len(loader.all_doors)}")
print(f"   Walls: {len(loader.all_walls)}")
print(f"   Total area: {loader.metadata.get('total_area', 0):.0f} m²")

## 🔗 Step 2: Building Connectivity Analysis

Let's analyze how rooms are connected and find critical circulation points:

In [ ]:
# Analyze building connectivity
from src.calculations.graph import create_circulation_graph, find_critical_circulation_points
from src.schemas import Project
import json

# Convert loader data to Project schema for analysis
def create_project_from_loader(loader):
    """Convert loader data to Project schema format."""
    project_data = {
        "metadata": loader.metadata,
        "levels": loader.levels
    }
    return Project(**project_data)

# Create project object
project = create_project_from_loader(loader)

print("🔗 Building Connectivity Analysis:")
print("=" * 60)

# Create circulation graph
graph = create_circulation_graph(project)
stats = graph.get_graph_statistics()

print(f"📊 Graph Statistics:")
print(f"   Nodes (rooms): {stats['total_nodes']}")
print(f"   Edges (doors): {stats['total_edges']}")
print(f"   Connected: {'✅ Yes' if stats['is_connected'] else '❌ No'}")
print(f"   Connected Components: {stats['connected_components']}")
print(f"   Average Connections per Room: {stats['average_degree']:.1f}")

# Find critical circulation points
critical_points = find_critical_circulation_points(project)

print(f"\n🎯 Critical Circulation Analysis:")
print(f"   Critical Rooms: {critical_points['critical_room_count']}")
print(f"   Connectivity Risk: {critical_points['connectivity_risk'].upper()}")

if critical_points['critical_rooms']:
    print(f"\n⚠️ Critical Rooms (removing these would disconnect building):")
    for room in critical_points['critical_rooms']:
        print(f"   🏠 {room['room_id']}: {room['room_name']}")
        print(f"      Connections: {room['connection_count']}")
        print(f"      Betweenness Score: {room['betweenness_score']:.3f}")
else:
    print("\n✅ No single points of failure found")

print(f"\n💡 Recommendations:")
for rec in critical_points['recommendations']:
    print(f"   • {rec}")

## 🚨 Step 3: Evacuation Route Analysis

Now let's analyze evacuation routes for all rooms:

In [ ]:
# Comprehensive evacuation analysis
from src.calculations.graph import find_all_evacuation_routes, validate_evacuation_compliance

print("🚨 Evacuation Route Analysis:")
print("=" * 60)

# Find all evacuation routes
all_routes = find_all_evacuation_routes(project)

print(f"📊 Evacuation Statistics:")
print(f"   Total Rooms: {all_routes['total_rooms']}")
print(f"   Average Distance: {all_routes['average_distance']:.1f} m")
print(f"   Compliance Rate: {all_routes['compliance_summary']['compliance_rate']:.1%}")

if all_routes['longest_route']:
    longest = all_routes['longest_route']
    print(f"\n🔍 Longest Evacuation Route:")
    print(f"   Room: {longest['room_id']}")
    print(f"   Distance: {longest['distance']:.1f} m")
    print(f"   Path: {' → '.join(longest['path'])}")
    print(f"   Accessible: {'✅ Yes' if longest['is_accessible'] else '❌ No'}")

if all_routes['shortest_route']:
    shortest = all_routes['shortest_route']
    print(f"\n⚡ Shortest Evacuation Route:")
    print(f"   Room: {shortest['room_id']}")
    print(f"   Distance: {shortest['distance']:.1f} m")

# Validate compliance
compliance = validate_evacuation_compliance(project, max_distance=25.0)

print(f"\n📋 Compliance Validation:")
print(f"   Overall Status: {compliance['overall_status']}")
print(f"   Compliant Rooms: {len(compliance['compliant_rooms'])}/{compliance['total_rooms']}")
print(f"   Non-Compliant: {len(compliance['non_compliant_rooms'])}")
print(f"   Inaccessible: {len(compliance['inaccessible_rooms'])}")

if compliance['non_compliant_rooms']:
    print(f"\n❌ Non-Compliant Rooms (exceed {compliance['max_distance_limit']}m):")
    for room in compliance['non_compliant_rooms'][:5]:  # Show first 5
        print(f"   🏠 {room['room_id']}: {room['distance']:.1f}m (excess: {room['excess_distance']:.1f}m)")

if compliance['inaccessible_rooms']:
    print(f"\n🚫 Inaccessible Rooms (no evacuation path):")
    for room in compliance['inaccessible_rooms']:
        print(f"   🏠 {room['room_id']}: {room['issue']}")

## 🚪 Step 4: Door Usage and Compliance Analysis

Let's analyze how doors are used and their compliance:

In [ ]:
# Door usage and compliance analysis
from src.calculations.graph import calculate_door_usage_analysis
from src.calculations.geometry import calculate_door_clear_width

print("🚪 Door Usage & Compliance Analysis:")
print("=" * 60)

# Analyze door usage in evacuation routes
door_usage = calculate_door_usage_analysis(project)

print(f"📊 Door Usage Statistics:")
print(f"   Total Usage Count: {door_usage['total_usage']}")
print(f"   High Usage Doors: {door_usage['usage_distribution']['high_usage']}")
print(f"   Medium Usage Doors: {door_usage['usage_distribution']['medium_usage']}")
print(f"   Unused Doors: {door_usage['usage_distribution']['unused']}")

if door_usage['most_used_door']:
    most_used_id, usage_count = door_usage['most_used_door']
    print(f"\n🔥 Most Used Door: {most_used_id} (used in {usage_count} routes)")

if door_usage['unused_doors']:
    print(f"\n💤 Unused Doors: {', '.join(door_usage['unused_doors'][:5])}")
    if len(door_usage['unused_doors']) > 5:
        print(f"   ... and {len(door_usage['unused_doors']) - 5} more")

# Door compliance analysis
print(f"\n📏 Door Compliance Analysis:")
compliant_doors = 0
non_compliant_doors = []
emergency_exits = 0

for door in loader.all_doors:
    width = door['width_mm']
    is_emergency = door.get('is_emergency_exit', False)
    
    if is_emergency:
        emergency_exits += 1
        min_width = 900  # Emergency exits need 900mm
    else:
        min_width = 800  # Regular doors need 800mm
    
    if width >= min_width:
        compliant_doors += 1
    else:
        non_compliant_doors.append({
            'id': door['id'],
            'width': width,
            'required': min_width,
            'deficit': min_width - width,
            'type': 'emergency' if is_emergency else 'regular'
        })

compliance_rate = compliant_doors / len(loader.all_doors) if loader.all_doors else 0

print(f"   Total Doors: {len(loader.all_doors)}")
print(f"   Emergency Exits: {emergency_exits}")
print(f"   Compliant Doors: {compliant_doors}/{len(loader.all_doors)} ({compliance_rate:.1%})")

if non_compliant_doors:
    print(f"\n❌ Non-Compliant Doors:")
    for door in non_compliant_doors[:3]:  # Show first 3
        print(f"   🚪 {door['id']}: {door['width']}mm (needs {door['deficit']}mm more)")
    if len(non_compliant_doors) > 3:
        print(f"   ... and {len(non_compliant_doors) - 3} more")
else:
    print(f"\n✅ All doors meet width requirements!")

## 👥 Step 5: Occupancy Load Analysis

Let's calculate occupancy loads and egress capacity:

In [ ]:
# Occupancy load analysis
from src.calculations.geometry import calculate_egress_capacity

print("👥 Occupancy Load Analysis:")
print("=" * 60)

# Analyze occupancy for each room
total_occupancy = 0
room_analysis = []

for room in loader.all_rooms:
    area = room['area']
    use_type = room.get('use', 'commercial')
    
    # Calculate occupancy based on use
    occupancy_factors = {
        "industrial": 0.02,    # 1 person per 50 sqm
        "warehouse": 0.02,    # 1 person per 50 sqm
        "office": 0.1,        # 1 person per 10 sqm
        "commercial": 0.1,    # 1 person per 10 sqm
        "assembly": 0.5,      # 1 person per 2 sqm
        "storage": 0.02,      # 1 person per 50 sqm
    }
    
    factor = occupancy_factors.get(use_type, 0.05)  # Default
    occupancy = max(1, int(area * factor))
    
    # Required egress width (5mm per person)
    required_width = occupancy * 5
    
    room_analysis.append({
        'id': room['id'],
        'name': room.get('name', room['id']),
        'area': area,
        'use': use_type,
        'occupancy': occupancy,
        'required_width': required_width
    })
    
    total_occupancy += occupancy

# Sort by occupancy (highest first)
room_analysis.sort(key=lambda x: x['occupancy'], reverse=True)

print(f"📊 Building Occupancy Summary:")
print(f"   Total Building Occupancy: {total_occupancy} people")
print(f"   Average per Room: {total_occupancy / len(room_analysis):.1f} people")
print(f"   Total Required Egress Width: {total_occupancy * 5:.0f} mm")

print(f"\n🏠 Top Rooms by Occupancy:")
print(f"{'Room ID':<8} {'Area (m²)':<10} {'Use':<12} {'Occupancy':<10} {'Egress Req.':<12}")
print("-" * 60)

for room in room_analysis[:10]:  # Show top 10
    print(f"{room['id']:<8} {room['area']:<10.0f} {room['use']:<12} {room['occupancy']:<10} {room['required_width']:<12.0f} mm")

# Find rooms with highest occupancy density
high_density_rooms = [r for r in room_analysis if r['occupancy'] / r['area'] > 0.1]

if high_density_rooms:
    print(f"\n⚠️ High Density Rooms (>0.1 people/m²):")
    for room in high_density_rooms[:3]:
        density = room['occupancy'] / room['area']
        print(f"   🏠 {room['id']}: {density:.2f} people/m² ({room['use']})")
else:
    print(f"\n✅ No high-density occupancy concerns found")

## 📊 Step 6: Spatial Relationships Analysis

Let's analyze how rooms relate to each other spatially:

In [ ]:
# Spatial relationships analysis
from src.calculations.graph import get_room_adjacency_list

print("📊 Spatial Relationships Analysis:")
print("=" * 60)

# Get room adjacency
adjacency = get_room_adjacency_list(project)

print(f"🔗 Room Connectivity Summary:")
print(f"   Total Rooms: {len(adjacency)}")

# Analyze connectivity patterns
connection_counts = [len(connections) for connections in adjacency.values()]
if connection_counts:
    avg_connections = sum(connection_counts) / len(connection_counts)
    max_connections = max(connection_counts)
    min_connections = min(connection_counts)
    
    print(f"   Average Connections per Room: {avg_connections:.1f}")
    print(f"   Range: {min_connections} - {max_connections} connections")

# Find hub rooms (highly connected)
hub_rooms = [(room_id, len(connections)) for room_id, connections in adjacency.items() 
             if len(connections) >= 3]
hub_rooms.sort(key=lambda x: x[1], reverse=True)

if hub_rooms:
    print(f"\n🎯 Hub Rooms (3+ connections):")
    for room_id, count in hub_rooms[:5]:
        # Find room name
        room_name = next((r.get('name', room_id) for r in loader.all_rooms if r['id'] == room_id), room_id)
        print(f"   🏠 {room_id}: {count} connections")
        
        # Show what it connects to
        connections = adjacency[room_id][:3]  # Show first 3
        if connections:
            print(f"      → Connected to: {', '.join(connections)}")
            if len(adjacency[room_id]) > 3:
                print(f"      → ... and {len(adjacency[room_id]) - 3} more")

# Find isolated rooms (no connections)
isolated_rooms = [room_id for room_id, connections in adjacency.items() if not connections]

if isolated_rooms:
    print(f"\n🏝️ Isolated Rooms (no connections):")
    for room_id in isolated_rooms[:5]:
        room_name = next((r.get('name', room_id) for r in loader.all_rooms if r['id'] == room_id), room_id)
        print(f"   🏠 {room_id}")
else:
    print(f"\n✅ All rooms are connected to the circulation network")

# Calculate building connectivity metrics
total_connections = sum(len(connections) for connections in adjacency.values())
connectivity_density = total_connections / (len(adjacency) * (len(adjacency) - 1)) if len(adjacency) > 1 else 0

print(f"\n📈 Building Connectivity Metrics:")
print(f"   Total Connections: {total_connections}")
print(f"   Connectivity Density: {connectivity_density:.3f}")
print(f"   Network Health: {'Good' if connectivity_density > 0.1 else 'Needs Improvement'}")

## 🎯 Step 7: Room-Specific Analysis Tool

Let's create a tool to analyze any specific room in detail:

In [ ]:
# Room-specific detailed analysis
from src.calculations.graph import calculate_room_connectivity_score

def analyze_specific_room(room_id: str):
    """Comprehensive analysis of a specific room."""
    
    print(f"🏠 Detailed Analysis: Room {room_id}")
    print("=" * 50)
    
    # Find room data
    room_data = next((r for r in loader.all_rooms if r['id'] == room_id), None)
    if not room_data:
        print(f"❌ Room {room_id} not found!")
        return
    
    # Basic room info
    print(f"📋 Basic Information:")
    print(f"   Name: {room_data.get('name', 'Unnamed')}")
    print(f"   Use: {room_data.get('use', 'Unknown')}")
    print(f"   Area: {room_data['area']:.1f} m²")
    print(f"   Level: {room_data.get('level', 'Unknown')}")
    
    # Connectivity analysis
    try:
        connectivity = calculate_room_connectivity_score(project, room_id)
        if 'error' not in connectivity:
            print(f"\n🔗 Connectivity Analysis:")
            print(f"   Direct Connections: {connectivity['direct_connections']}")
            print(f"   Reachable Rooms: {connectivity['reachable_rooms']}/{connectivity['total_rooms']}")
            print(f"   Reachability Score: {connectivity['reachability_score']:.2f}")
            print(f"   Average Distance to Others: {connectivity['average_distance']:.1f} m")
            print(f"   Connectivity Grade: {connectivity['connectivity_grade'].upper()}")
    except Exception as e:
        print(f"\n⚠️ Connectivity analysis unavailable: {str(e)}")
    
    # Evacuation analysis
    try:
        graph = create_circulation_graph(project)
        evacuation = graph.calculate_egress_distance(room_id)
        
        if 'error' not in evacuation:
            print(f"\n🚨 Evacuation Analysis:")
            print(f"   Distance to Exit: {evacuation['distance']:.1f} m")
            print(f"   Path: {' → '.join(evacuation['path'])}")
            print(f"   Accessible Route: {'✅ Yes' if evacuation['is_accessible'] else '❌ No'}")
            
            # Compliance check
            if evacuation['distance'] <= 25:
                print(f"   Compliance: ✅ Within 25m limit")
            elif evacuation['distance'] <= 35:
                print(f"   Compliance: ⚠️ Exceeds 25m but within 35m")
            else:
                print(f"   Compliance: ❌ Exceeds 35m limit")
    except Exception as e:
        print(f"\n⚠️ Evacuation analysis unavailable: {str(e)}")
    
    # Occupancy analysis
    area = room_data['area']
    use_type = room_data.get('use', 'commercial')
    
    occupancy_factors = {
        "industrial": 0.02, "warehouse": 0.02, "office": 0.1,
        "commercial": 0.1, "assembly": 0.5, "storage": 0.02
    }
    
    factor = occupancy_factors.get(use_type, 0.05)
    occupancy = max(1, int(area * factor))
    required_egress = occupancy * 5  # 5mm per person
    
    print(f"\n👥 Occupancy Analysis:")
    print(f"   Calculated Occupancy: {occupancy} people")
    print(f"   Occupancy Factor: {factor} people/m²")
    print(f"   Required Egress Width: {required_egress} mm")
    print(f"   Density: {occupancy/area:.3f} people/m²")

# Example: Analyze a specific room
print("🔍 Room Analysis Tool Demo:")
print("\nAvailable rooms:", [r['id'] for r in loader.all_rooms[:5]], "...")

# Analyze the first room as an example
if loader.all_rooms:
    example_room = loader.all_rooms[0]['id']
    print(f"\n📝 Example analysis for room: {example_room}")
    analyze_specific_room(example_room)
    
    print(f"\n💡 To analyze any room, call: analyze_specific_room('ROOM_ID')")

## 📈 Step 8: Performance Summary Dashboard

Let's create a comprehensive building performance summary:

In [ ]:
# Building performance dashboard
print("📈 BUILDING PERFORMANCE DASHBOARD")
print("=" * 70)
print(f"Project: {loader.metadata.get('project_name')}")
print(f"Analysis Date: {loader.metadata.get('created_date', '')[:10]}")
print("=" * 70)

# Overall metrics
print(f"\n🏢 BUILDING OVERVIEW:")
print(f"   Levels: {len(loader.levels)}")
print(f"   Total Area: {loader.metadata.get('total_area', 0):.0f} m²")
print(f"   Rooms: {len(loader.all_rooms)}")
print(f"   Doors: {len(loader.all_doors)}")
print(f"   Walls: {len(loader.all_walls)}")

# Safety metrics
try:
    compliance = validate_evacuation_compliance(project)
    print(f"\n🚨 SAFETY PERFORMANCE:")
    print(f"   Evacuation Compliance: {compliance['compliance_rate']:.1%}")
    print(f"   Compliant Rooms: {len(compliance['compliant_rooms'])}/{compliance['total_rooms']}")
    print(f"   Issues: {len(compliance['non_compliant_rooms'])} rooms exceed distance limits")
    print(f"   Inaccessible: {len(compliance['inaccessible_rooms'])} rooms")
    
    safety_grade = "A" if compliance['compliance_rate'] >= 0.95 else "B" if compliance['compliance_rate'] >= 0.85 else "C" if compliance['compliance_rate'] >= 0.75 else "D"
    print(f"   Safety Grade: {safety_grade}")
except Exception as e:
    print(f"\n🚨 SAFETY PERFORMANCE: Analysis unavailable ({str(e)})")

# Circulation metrics
try:
    critical_points = find_critical_circulation_points(project)
    print(f"\n🔗 CIRCULATION PERFORMANCE:")
    print(f"   Critical Rooms: {critical_points['critical_room_count']}")
    print(f"   Connectivity Risk: {critical_points['connectivity_risk'].upper()}")
    
    circulation_grade = "A" if critical_points['connectivity_risk'] == 'low' else "B" if critical_points['connectivity_risk'] == 'medium' else "C"
    print(f"   Circulation Grade: {circulation_grade}")
except Exception as e:
    print(f"\n🔗 CIRCULATION PERFORMANCE: Analysis unavailable ({str(e)})")

# Door compliance
emergency_exits = sum(1 for d in loader.all_doors if d.get('is_emergency_exit', False))
compliant_doors = sum(1 for d in loader.all_doors 
                     if d['width_mm'] >= (900 if d.get('is_emergency_exit', False) else 800))
door_compliance_rate = compliant_doors / len(loader.all_doors) if loader.all_doors else 0

print(f"\n🚪 DOOR COMPLIANCE:")
print(f"   Total Doors: {len(loader.all_doors)}")
print(f"   Emergency Exits: {emergency_exits}")
print(f"   Compliance Rate: {door_compliance_rate:.1%}")
print(f"   Non-Compliant: {len(loader.all_doors) - compliant_doors}")

door_grade = "A" if door_compliance_rate >= 0.95 else "B" if door_compliance_rate >= 0.85 else "C" if door_compliance_rate >= 0.75 else "D"
print(f"   Door Grade: {door_grade}")

# Occupancy metrics
total_occupancy = sum(max(1, int(r['area'] * 0.05)) for r in loader.all_rooms)  # Simplified
avg_density = total_occupancy / loader.metadata.get('total_area', 1)

print(f"\n👥 OCCUPANCY ANALYSIS:")
print(f"   Total Capacity: {total_occupancy} people")
print(f"   Average Density: {avg_density:.3f} people/m²")
print(f"   Required Egress: {total_occupancy * 5:.0f} mm total width")

# Overall building score
try:
    scores = []
    if 'safety_grade' in locals():
        safety_score = {"A": 95, "B": 85, "C": 75, "D": 65}.get(safety_grade, 60)
        scores.append(safety_score)
    
    if 'circulation_grade' in locals():
        circulation_score = {"A": 95, "B": 85, "C": 75}.get(circulation_grade, 65)
        scores.append(circulation_score)
    
    door_score = {"A": 95, "B": 85, "C": 75, "D": 65}.get(door_grade, 60)
    scores.append(door_score)
    
    overall_score = sum(scores) / len(scores) if scores else 0
    
    print(f"\n🎯 OVERALL BUILDING SCORE: {overall_score:.0f}/100")
    
    if overall_score >= 90:
        print(f"   Rating: ⭐⭐⭐⭐⭐ EXCELLENT")
    elif overall_score >= 80:
        print(f"   Rating: ⭐⭐⭐⭐ GOOD")
    elif overall_score >= 70:
        print(f"   Rating: ⭐⭐⭐ ACCEPTABLE")
    else:
        print(f"   Rating: ⭐⭐ NEEDS IMPROVEMENT")
        
except Exception as e:
    print(f"\n🎯 OVERALL SCORE: Unable to calculate ({str(e)})")

print(f"\n" + "=" * 70)
print("✅ Performance analysis complete!")
print("\n💡 These are the tools our compliance agent uses to analyze buildings automatically.")

## 🎯 Summary: Agent Computational Tools

This notebook demonstrated the **practical computational tools** our compliance agent uses:

### 🔧 **Core Analysis Tools**:
1. **Building Connectivity Analysis** - Identifies critical circulation points and connection patterns
2. **Evacuation Route Analysis** - Finds optimal paths and validates compliance against regulations
3. **Door Usage Analysis** - Tracks which doors are most critical for circulation
4. **Occupancy Load Calculations** - Determines capacity and egress requirements
5. **Spatial Relationship Analysis** - Maps how rooms connect and relate to each other
6. **Room-Specific Analysis** - Deep dive into any room's performance metrics

### 🎪 **Key Benefits**:
- **Automated Analysis**: No manual calculations needed
- **Real Building Data**: Works with actual IFC extractions
- **Compliance Focus**: Built-in regulatory requirements
- **Agent-Ready**: All tools available for autonomous use

### 🚀 **For the Agent**:
The agent can call these tools autonomously to:
- Answer questions about building performance
- Identify compliance issues
- Suggest improvements
- Generate comprehensive reports

**Next**: These tools integrate with the RAG system and agent reasoning for complete autonomous analysis!